In [3]:
import pandas as pd

train_dataset = pd.read_csv('/content/drive/MyDrive/Datavidia/train_dataset_advanced.csv', index_col=0)
Y_training_df = train_dataset["main-label"]
# Drop 'label' column
X_training_df = train_dataset.drop(labels = ['main-label','main-id'],axis = 1) 
print('Y_training shape', Y_training_df.shape)
print('X_training shape', X_training_df.shape) 

Y_training shape (14856,)
X_training shape (14856, 15408)


In [4]:
import numpy as np

X_training_array = X_training_df.values
X_training = np.reshape(X_training_array,(14856,48,107,3))
Y_training = Y_training_df.values

In [5]:
# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_training, Y_training, test_size = 0.3, random_state=2)
print("x_train shape",X_train.shape)
print("x_val shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_val shape",Y_val.shape)

x_train shape (10399, 48, 107, 3)
x_val shape (4457, 48, 107, 3)
y_train shape (10399,)
y_val shape (4457,)


In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers

model = Sequential()
model.add(Conv2D(128, (3,3), activation='relu', input_shape=(48, 107, 3)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

opt = optimizers.Adam(lr=0.0001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 46, 105, 128)      3584      
_________________________________________________________________
flatten_13 (Flatten)         (None, 618240)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 618241    
Total params: 621,825
Trainable params: 621,825
Non-trainable params: 0
_________________________________________________________________
None


In [86]:
epochs = 50  # for better result increase the epochs
batch_size = 16

# Fit the model
history = model.fit(X_train, Y_train, 
                    epochs = epochs, validation_data = (X_val,Y_val), batch_size=batch_size)

Epoch 1/50
650/650 [==============================] - 5s 7ms/step - loss: 0.3641 - accuracy: 0.8687 - val_loss: 0.2066 - val_accuracy: 0.9188
Epoch 2/50
650/650 [==============================] - 4s 7ms/step - loss: 0.1730 - accuracy: 0.9385 - val_loss: 0.1791 - val_accuracy: 0.9421
Epoch 3/50
650/650 [==============================] - 4s 7ms/step - loss: 0.1410 - accuracy: 0.9527 - val_loss: 0.1729 - val_accuracy: 0.9462
Epoch 4/50
650/650 [==============================] - 4s 7ms/step - loss: 0.1250 - accuracy: 0.9600 - val_loss: 0.1813 - val_accuracy: 0.9435
Epoch 5/50
650/650 [==============================] - 4s 7ms/step - loss: 0.1074 - accuracy: 0.9648 - val_loss: 0.1779 - val_accuracy: 0.9462
Epoch 6/50
650/650 [==============================] - 4s 7ms/step - loss: 0.0940 - accuracy: 0.9715 - val_loss: 0.1816 - val_accuracy: 0.9459
Epoch 7/50
650/650 [==============================] - 4s 7ms/step - loss: 0.0915 - accuracy: 0.9721 - val_loss: 0.1852 - val_accuracy: 0.9446
Epoch 

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='b', label="validation loss", linestyle=':')
plt.plot(history.history['loss'], color='b', label="loss", linestyle='-')
plt.plot(history.history['val_accuracy'], color='r', label="validation accuracy", linestyle=':')
plt.plot(history.history['accuracy'], color='r', label="accuracy", linestyle='-')
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Test Section

In [ ]:
import pandas as pd

test_dataset = pd.read_csv('/content/drive/MyDrive/Datavidia/test_dataset_advanced.csv', index_col=0)
# Drop 'label' column
X_testing_df = test_dataset.drop(labels = ['main-id'],axis = 1)
print('X_testing shape', X_testing_df.shape) 

In [ ]:
import numpy as np

X_testing_array = X_testing_df.values
X_testing = np.reshape(X_testing_array,(3714,48,107,3))

In [ ]:
from copy import deepcopy
import numpy as np

def binary_classifier(label):
  temp = deepcopy(label)
  for i in temp:
    if i[0] > 0.5:
      i[0] = 1
    else:
      i[0] = 0
  return temp

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(X_testing)
Y_pred_advanced = binary_classifier(Y_pred).astype(int)
Y_pred_advanced = Y_pred_advanced.reshape(-1)
print(Y_pred_advanced)

In [ ]:
temp1 = test_dataset['main-id'].values
test_result = list(zip(list(temp1),list(Y_pred_advanced)))
test_result_df = pd.DataFrame(test_result,columns=['review_id','category'])
test_result_df

In [ ]:
test_result_df.to_csv('/content/drive/MyDrive/Datavidia/test_result-CNN.csv', index=False)